In [1]:
import sys
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
sys.path.insert(0, os.path.abspath('../../..'))
from g4l.estimators import BIC
from g4l.estimators import SMC
import g4l.display
from g4l.estimators import Prune
from g4l.models import ContextTree
from g4l.models import integrity
from g4l.data import Sample

## Comparing the mutual ocurrence of trees between methods

In [2]:
results_folder = os.path.abspath('../../simulation_study/bb/results')
methods = ['SeqROCTM', 'prune', 'smc']
methods = ['smc']
models = ['model1']
sizes = [5000]
correct_model = '000 1 10 100'

import warnings

#import pandas as pd
from pandas.core.common import SettingWithCopyWarning

warnings.simplefilter(action="ignore", category=SettingWithCopyWarning)


def get_results(method, model_name, size):
    file = '%s/%s/%s_%s.csv' % (results_folder, method, model_name, size)
    df = pd.read_csv(file)
    df.tree = df.tree.astype(str).map(lambda t: ' '.join(sorted(t.split())))
    return df

def get_champion_trees(df1, df2, sample_idx):
    trees1 = df1[df1.sample_idx == sample_idx]
    trees2 = df2[df2.sample_idx == sample_idx]
    
    #trees2.tree = trees2.tree.astype(str).map(lambda t: ' '.join(sorted(t.split())))
    trees2['match'] = trees2.tree.isin(trees1.tree).astype(int)
    trees1['match'] = trees1.tree.isin(trees2.tree).astype(int)
    return trees1, trees2

def compare(method1, method2, model, size):
    df = pd.DataFrame(columns=['sample_idx', 'precision', 'recall'])
    df1 = get_results(method1, model, size)
    df2 = get_results(method2, model, size)
    for sample_idx in df1.sample_idx.unique():
        trees1, trees2 = get_champion_trees(df1, df2, sample_idx)
        precision = trees2['match'].sum()/len(trees2)
        recall = trees1['match'].sum()/len(trees1)
        df.loc[len(df)] = [int(sample_idx), precision, recall]
    return df, df1, df2

In [3]:
#df, df1, df2 = compare('SeqROCTM', 'smc', 'model1', 1000)
#df.mean()

In [4]:
#x = get_results('SeqROCTM', 'model1', 5000)
#x.head(30)

In [6]:
method_name = 'smc'
model = 'model1'
size = 5000
#df, df1, df2 = get_results(method_name, model, size)
df, df1, df2 = compare('smc', 'SeqROCTM', model, size)
prec_rec = df[['precision', 'recall']].mean()
dfr = df1[df1.tree=='000 1 10 100']
dfr2 = df2[df2.tree=='000 1 10 100']
correct_tree_occurrence = len(dfr)
found = dfr.opt.sum()
found2 = dfr2.opt.sum()
print("Method:", method_name)
print("Size:", size)
print("Model:", model)
print("Occurrences of correct tree:", correct_tree_occurrence)
print("Found by python version:", found2)
print("Found by matlab version:", found)
print("Precision: ", prec_rec.precision)
print("Recall: ", prec_rec.recall)

FileNotFoundError: [Errno 2] No such file or directory: '/home/arthur/Documents/Neuromat/projects/SMC/smallest_maximizer_criterion/examples/simulation_study/bb/results/SeqROCTM/model1_5000.csv'

In [6]:
dfr2['opt_smc'] = dfr2.opt 
seq_opt = dfr.set_index('sample_idx')[['opt']]
smc_opt = dfr2.set_index('sample_idx')[['opt_smc']]
seq_opt.join(smc_opt)

,opt,opt_smc
sample_idx,,
0,1,1
1,0,0
2,0,0
3,1,1
5,1,1
...,...,...
93,0,0
95,0,0
96,0,0
